In [13]:
import boto3
import pprint
import json
from io import StringIO
import pandas as pd

## Part 1

Using the dataframe from the population data API (Part 2), generate the mean and the standard deviation of the US population across the years [2013, 2018] inclusive.

In [14]:
s3 = boto3.client("s3")
acs_data = s3.get_object(Bucket='ryan-rearc', Key='acs_data.json')['Body'].read()
pr_data = s3.get_object(Bucket='ryan-rearc', Key='bls_data/pr.data.0.Current')['Body'].read()

In [77]:
acs_dict = json.loads(acs_data)['data']
acs_df = pd.DataFrame.from_records(acs_dict)
acs_df['Year'] = pd.to_numeric(acs_df['Year'])
acs_df.columns = x.map(lambda x: x.replace(' ', '_').lower())
acs_df

,id_nation,nation,id_year,year,population,slug_nation
0,01000US,United States,2020,2020,326569308,united-states
1,01000US,United States,2019,2019,324697795,united-states
2,01000US,United States,2018,2018,322903030,united-states
3,01000US,United States,2017,2017,321004407,united-states
4,01000US,United States,2016,2016,318558162,united-states
5,01000US,United States,2015,2015,316515021,united-states
6,01000US,United States,2014,2014,314107084,united-states
7,01000US,United States,2013,2013,311536594,united-states


In [78]:
population = acs_df[acs_df['year'] <= 2018]['population']
print('Population mean is: ' + f'{population.mean():,}')
print('Population standard deviation is: ' + f'{population.std():,}')

Population mean is: 317,437,383.0
Population standard deviation is: 4,257,089.5415293295


## Part 2

Using the dataframe from the time-series (Part 1), For every series_id, find the best year: the year with the max/largest sum of "value" for all quarters in that year. Generate a report with each series id, the best year for that series, and the summed value for that year

In [67]:
s = str(pr_data, 'utf-8')
pr_df = pd.read_csv(StringIO(s), sep='\t')
pr_df.head(5)

,series_id,year,period,value,footnote_codes
0,PRS30006011,1995,Q01,2.6,NaN
1,PRS30006011,1995,Q02,2.1,NaN
2,PRS30006011,1995,Q03,0.9,NaN
3,PRS30006011,1995,Q04,0.1,NaN
4,PRS30006011,1995,Q05,1.4,NaN


In [82]:
pr_df.columns = pr_df.columns.str.replace(' ', '')
# Upon testing, it doesn't appear there are any values in period that need to be trimmed, but I had already written the code, and I find it good practice to trim data as a first step
pr_df['period'] = pr_df['period'].str.strip()
pr_df['series_id'] = pr_df['series_id'].str.strip()

In [83]:
pr_df.columns

Index(['series_id', 'year', 'period', 'value', 'footnote_codes'], dtype='object')

In [105]:
id_year = pr_df.groupby(by=['series_id', 'year']).sum('value').reset_index()
id_year

,series_id,year,value
0,PRS30006011,1995,7.100
1,PRS30006011,1996,-0.500
2,PRS30006011,1997,4.400
3,PRS30006011,1998,4.200
4,PRS30006011,1999,-7.700
...,...,...,...
7801,PRS88003203,2018,551.596
7802,PRS88003203,2019,564.447
7803,PRS88003203,2020,567.794
7804,PRS88003203,2021,580.551


In [106]:
id_year.loc[id_year.groupby('series_id')['value'].idxmax()]

,series_id,year,value
27,PRS30006011,2022,13.200
54,PRS30006012,2021,13.100
59,PRS30006013,1998,733.348
99,PRS30006021,2010,17.700
127,PRS30006022,2010,12.500
...,...,...,...
7673,PRS88003192,2002,285.200
7720,PRS88003193,2021,647.437
7734,PRS88003201,2007,21.700
7776,PRS88003202,2021,24.000


In [120]:
# Checking for duplicates, because for series_id if there are any years with the same sum(value), the above code would return both years (I don't know what the desired behavior would in that case)
id_year['series_id'].nunique()

282

## Part 3

Using both dataframes from Part 1 and Part 2, generate a report that will provide the value for series_id = PRS30006032 and period = Q01 and the population for that given year (if available in the population dataset)

In [110]:
selected_quarter_df = pr_df[(pr_df['series_id'] == 'PRS30006032') & (pr_df['period'] == 'Q01')]
selected_quarter_df

,series_id,year,period,value,footnote_codes
966,PRS30006032,1995,Q01,0.0,NaN
971,PRS30006032,1996,Q01,-4.4,NaN
976,PRS30006032,1997,Q01,2.7,NaN
981,PRS30006032,1998,Q01,1.0,NaN
986,PRS30006032,1999,Q01,-4.1,NaN
991,PRS30006032,2000,Q01,0.1,NaN
996,PRS30006032,2001,Q01,-6.0,NaN
1001,PRS30006032,2002,Q01,-7.0,NaN
1006,PRS30006032,2003,Q01,-5.7,NaN
1011,PRS30006032,2004,Q01,2.4,NaN


In [121]:
cols_to_select = ['series_id', 'year', 'period', 'value', 'population']
joined = selected_quarter_df.merge(acs_df, on='year')
joined[cols_to_select]

,series_id,year,period,value,population
0,PRS30006032,2013,Q01,1.2,311536594
1,PRS30006032,2014,Q01,0.0,314107084
2,PRS30006032,2015,Q01,-1.7,316515021
3,PRS30006032,2016,Q01,-1.8,318558162
4,PRS30006032,2017,Q01,1.0,321004407
5,PRS30006032,2018,Q01,-0.2,322903030
6,PRS30006032,2019,Q01,-2.5,324697795
7,PRS30006032,2020,Q01,-5.8,326569308
